In [2]:
import cv2
import os
import numpy as np

def augment_images_in_directory(directory, aug_dir):
    # Create directories for augmented images if they don't exist
    if not os.path.exists(aug_dir):
        os.makedirs(aug_dir)

    # List all image files in the directory
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    num_images = 20
    for image_file in image_files:
        if (num_images > 0):
            num_images -= 1
        else:
            return
        image_path = os.path.join(directory, image_file)
        image = cv2.imread(image_path)
        filename, ext = os.path.splitext(image_file)

        # Rotation
        for angle in [30, 60, 90, 120, 150]:
            rotated = rotate_image(image, angle)
            save_image(rotated, aug_dir, filename + "_rotated_" + str(angle) + ext)

        # Resizing
        for scale in [0.5, 0.75, 1.25, 1.5]:
            resized = resize_image(image, scale)
            save_image(resized, aug_dir, filename + "_resized_" + str(int(scale*100)) + ext)

        # Horizontal flipping
        flipped = cv2.flip(image, 1)
        save_image(flipped, aug_dir, filename + "_flipped" + ext)

        # Shearing
        sheared = shear_image(image)
        save_image(sheared, aug_dir, filename + "_sheared" + ext)

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def resize_image(image, scale):
    resized = cv2.resize(image, None, fx=scale, fy=scale)
    return resized

def shear_image(image):
    rows, cols, _ = image.shape
    shear_factor = 0.2  # Adjust the shear factor as needed
    shear_matrix = np.array([[1, shear_factor, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, shear_matrix, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return sheared

def save_image(image, directory, filename):
    save_path = os.path.join(directory, filename)
    cv2.imwrite(save_path, image)


In [3]:
disease_images_dir_list = [
    './PlantVillage/train/Apple___Apple_scab',
    './PlantVillage/train/Apple___Black_rot',
    './PlantVillage/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    './PlantVillage/train/Corn_(maize)___Common_rust_',
    './PlantVillage/train/Corn_(maize)___Northern_Leaf_Blight', 
    './PlantVillage/train/Grape___Black_rot',
    './PlantVillage/train/Grape___Esca_(Black_Measles)',
    './PlantVillage/train/Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    './PlantVillage/train/Pepper,_bell___Bacterial_spot',
    './PlantVillage/train/Strawberry___Leaf_scorch',
    './PlantVillage/train/Tomato___Bacterial_spot',
    './PlantVillage/train/Tomato___Early_blight', 
    './PlantVillage/train/Tomato___Late_blight', 
    './PlantVillage/train/Tomato___Leaf_Mold',
    './PlantVillage/train/Tomato___Septoria_leaf_spot',
    './PlantVillage/train/Tomato___Spider_mites Two-spotted_spider_mite',
    './PlantVillage/train/Tomato___Target_Spot',
    './PlantVillage/train/Tomato___Tomato_mosaic_virus',
    './PlantVillage/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
]

In [4]:
for diseased_folder in disease_images_dir_list:
    index = diseased_folder.rfind("/")
    diseased_folder_name = diseased_folder[index+1:]
    print("Performing Traditional Augmentation for " + str(diseased_folder))
    print(diseased_folder)
    augment_images_in_directory(diseased_folder, './traditional-augmented/'+str(diseased_folder_name))

Performing Traditional Augmentation for ./PlantVillage/train/Apple___Apple_scab
./PlantVillage/train/Apple___Apple_scab
Performing Traditional Augmentation for ./PlantVillage/train/Apple___Black_rot
./PlantVillage/train/Apple___Black_rot
Performing Traditional Augmentation for ./PlantVillage/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
./PlantVillage/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Performing Traditional Augmentation for ./PlantVillage/train/Corn_(maize)___Common_rust_
./PlantVillage/train/Corn_(maize)___Common_rust_
Performing Traditional Augmentation for ./PlantVillage/train/Corn_(maize)___Northern_Leaf_Blight
./PlantVillage/train/Corn_(maize)___Northern_Leaf_Blight
Performing Traditional Augmentation for ./PlantVillage/train/Grape___Black_rot
./PlantVillage/train/Grape___Black_rot
Performing Traditional Augmentation for ./PlantVillage/train/Grape___Esca_(Black_Measles)
./PlantVillage/train/Grape___Esca_(Black_Measles)
Performing Traditional Augme

In [5]:
import os
import random
import shutil

# Define paths
input_folder = "../traditional-augmentation/traditional-augmented"
train_folder = "../traditional-augmentation/traditional-augmented-div/train"
val_folder = "../traditional-augmentation/traditional-augmented-div/val"

# Create train and validation folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Function to divide images into train and validation sets
def split_dataset(input_folder, train_folder, val_folder, split_ratio=0.8):
    for root, dirs, files in os.walk(input_folder):
        for dir_name in dirs:
            # Create corresponding folders in train and validation sets
            train_dir = os.path.join(train_folder, root[len(input_folder)+1:], dir_name)
            val_dir = os.path.join(val_folder, root[len(input_folder)+1:], dir_name)
            os.makedirs(train_dir, exist_ok=True)
            os.makedirs(val_dir, exist_ok=True)
            
            # Get list of all image files in the subfolder
            image_files = [f for f in os.listdir(os.path.join(root, dir_name)) if f.endswith(('.jpg', '.jpeg', '.png', '.JPG'))]
            
            # Shuffle the list of image files
            random.shuffle(image_files)
            
            # Calculate number of images for train and validation sets
            num_images = len(image_files)
            num_train = int(split_ratio * num_images)
            num_val = num_images - num_train
            
            # Split the image files into train and validation sets
            train_images = image_files[:num_train]
            val_images = image_files[num_train:]
            
            # Copy train images to train folder
            for image in train_images:
                src = os.path.join(root, dir_name, image)
                dst = os.path.join(train_dir, image)
                shutil.copyfile(src, dst)
            
            # Copy validation images to validation folder
            for image in val_images:
                src = os.path.join(root, dir_name, image)
                dst = os.path.join(val_dir, image)
                shutil.copyfile(src, dst)

# Split the dataset into train and validation sets
split_dataset(input_folder, train_folder, val_folder)

print("Dataset split into train and validation sets successfully.")

Dataset split into train and validation sets successfully.


In [7]:
import os
import shutil

def copy_folder_contents(source_dir, dest_dir):
    for root, dirs, files in os.walk(source_dir):
        # Get the relative path from the source directory
        relative_path = os.path.relpath(root, source_dir)
        
        # Construct the corresponding destination directory
        dest_path = os.path.join(dest_dir, relative_path)
        
        # Ensure the destination directory exists
        os.makedirs(dest_path, exist_ok=True)
        
        # Copy files
        for file in files:
            src_file = os.path.join(root, file)
            dest_file = os.path.join(dest_path, file)
            shutil.copy(src_file, dest_file)

source_folder = "../traditional-augmentation/traditional-augmented-div/val"
destination_folder = "../augmentation-validation/traditional-augmented/PlantVillage/val"

copy_folder_contents(source_folder, destination_folder)